In [1]:
(ns curve-fitting
  (:refer-clojure :exclude [get contains? keys empty? dissoc assoc get-in
                            map reduce replicate apply])
  (:require [clojure.pprint :refer [pprint]]
            [metaprob.syntax :refer [gen]]
            [metaprob.compound :refer :all] ;; get, contains?, keys, empty?, dissoc, assoc, get-in
            [metaprob.builtin :refer :all] ;; map, reduce, replicate, apply
            [metaprob.trace :refer :all]
            [metaprob.prelude :refer :all]
            [metaprob.distributions :refer :all]
            [metaprob.tutorial.jupyter :refer :all]))

In [19]:
(use 'metaprob.inference :reload-all)

In [37]:
(def generate-curve
  (gen {:tracing-with t} []
    (let [degree (t "degree" uniform-sample [0 1 2 3])
          coeffs (map (gen [i] [i (t `("coeffs" ~i) gaussian 0 (/ 5 (inc degree)))])
                      (range (inc degree)))]
    (gen [x]
      (apply + 
        (map (gen [[n c]] (* c (expt x n))) coeffs))))))

;; (define add-noise-to-curve
;;   (gen [curve]
;;     ;; Sample noise hyperparameters
;;     (define inlier-noise (uniform 0.1 1)) ;(gamma 2 1))
;;     (define outlier-noise (uniform 1 5)) ; (gamma 1 1))
;;     (define prob-outlier (uniform 0 0.5)) ; (beta 1 10)
;;     ;; Return a version of the curve that
;;     ;; adds noise.
;;     (gen [x]
;;       (with-explicit-tracer t
;;         (define is-outlier (t "is-outlier" flip prob-outlier))
;;         (define mean (if is-outlier 0 (curve x)))
;;         (define noise (if is-outlier outlier-noise inlier-noise))
;;         (t "y" gaussian mean noise)))))

(def add-noise-to-curve
  (gen {:tracing-with t} [curve]
    ;; Sample noise hyperparameters
    (let [inlier-noise (t "inlier-noise" gamma 1 0.5)
          outlier-noise (t "outlier-noise" gamma 4 2)
          prob-outlier (t "prob-outlier" beta 5 50)]
      (gen {:tracing-with t} [x]
        (let [[mean noise]
               (if (t "outlier?" flip prob-outlier)
                 [0 outlier-noise]
                 [(curve x) inlier-noise])]
          (t "y" gaussian mean noise))))))


(def curve-model
  (gen {:tracing-with t} [xs]
     (let [underlying-curve (t "underlying-curve" generate-curve)
           noisy-curve (t "noise-params" add-noise-to-curve underlying-curve)]
       (map-indexed (gen [i x] (t `("data" ~i) noisy-curve x)) xs))))

#'curve-fitting/curve-model

In [42]:
(def make-observation-trace
  (gen [ys]
    {"data" (into {} (map-indexed (gen [i y] [i {"y" {:value y}}]) ys))}))

#'curve-fitting/make-observation-trace

In [43]:
(def count-points
  (gen [tr] (count (keys (trace-subtrace tr "data")))))
(def get-ys
  (gen [tr]
    (map (gen [i] (trace-value tr `("data" ~i "y"))) (range (count-points tr)))))

#'curve-fitting/get-ys

In [44]:
(def x-min -5)
(def x-max 5)
(def y-min -10)
(def y-max 10)
(def point-count 10)

(def x-range (- x-max x-min))
(def x-interval (/ x-range (- point-count 1)))
(def y-range (- y-max y-min))
(def y-interval (/ y-range (- point-count 1)))
(def xs
  (map-indexed (gen [i interval] (+ -5 (* interval i))) (repeat point-count x-interval)))

#'curve-fitting/xs

In [45]:
(def ys-linear
  (map-indexed (gen [i x] (+ y-min (* y-interval i))) xs))

#'curve-fitting/ys-linear

In [46]:
(def ys-bump
    (map (gen [x] (- x 10)) [6 7 6 5 4 3 2 19 2 3]))

#'curve-fitting/ys-bump

In [47]:
(def y-outlier (/ (- y-max y-min) 10))
(def ys-linear-outlier
    (map-indexed (gen [i y] (cond (= i 1) (+ (nth ys-linear 9) y-outlier)
                                  (= i 8) (- (nth ys-linear 0) y-outlier)
                                  true y)) ys-linear))

#'curve-fitting/ys-linear-outlier

In [48]:
(def ys-quadr
    (map-indexed (gen [i x] 
                   (let [real-value (+ -1 (* 0.2 x) (* 0.3 (expt x 2)))]
                     (if (= i 4) (+ real-value 4) real-value)))
        xs))

#'curve-fitting/ys-quadr

In [49]:
(def func-from-tr 
  (gen [tr]
    (nth (infer-and-score :procedure generate-curve :intervention-trace (trace-subtrace tr "underlying-curve")) 0)))

#'curve-fitting/func-from-tr

In [50]:
(def get-is-traces
  (gen [ys]
    (replicate 10 (gen [] (importance-resampling curve-model [xs] (make-observation-trace ys) 1000)))))

#'curve-fitting/get-is-traces

In [51]:
(def xs-simple '(-1 0 1))
(def ys (curve-model xs-simple))

#'curve-fitting/ys

In [52]:
(infer-and-score 
    :procedure curve-model
    :inputs ['(0 1)]
    :observation-trace (make-observation-trace '(1.70 -1.46)))

[(1.7 -1.46) {"underlying-curve" {"degree" {:value 0}, "coeffs" {0 {:value -3.2201946536753145}}}, "noise-params" {"inlier-noise" {:value 0.2679998758449996}, "outlier-noise" {:value 4.797971447834098}, "prob-outlier" {:value 0.031424674366601235}}, "data" {0 {"outlier?" {:value false}, "y" {:value 1.7}}, 1 {"outlier?" {:value false}, "y" {:value -1.46}}}} -189.29836068570552]

In [53]:
(plot-trace 
    {"underlying-curve" {"degree" {:value 1}, "coeffs" {0 {:value -2.47}, 1 {:value 3.96}}}, 
 "noise-params" {"inlier-noise" {:value 0.75}, 
 "outlier-noise" {:value 9.06}, 
 "prob-outlier" {:value 0.06}}, 
 "data" {0 {"is-outlier" {:value false}, "y" {:value 1.7}}, 
 1 {"is-outlier" {:value false}, "y" {:value -1.46}}}} 350 480)

In [54]:
(clojure.core/require '[clojure.data.json :as json])


In [56]:
(def serialize-curve-trace
  (gen [tr]
    {:inlier-noise (trace-value tr '("noise-params" "inlier-noise"))
     :outlier-noise (trace-value tr '("noise-params" "outlier-noise"))
     :prob-outlier (trace-value tr '("noise-params" "prob-outlier"))
     :outlier-designations (map (gen [i] (trace-value tr `("data" ~i "is-outlier"))) (range (count xs)))
     :degree (trace-value tr '("underlying-curve" "degree"))
     :coeffs (map (gen [i] (trace-value tr `("underlying-curve" "coeffs" ~i))) (range (+ 1 (trace-value tr '("underlying-curve" "degree")))))}))

#'curve-fitting/serialize-curve-trace

In [34]:
;(define is-traces-linear (get-is-traces ys-linear))
(map #(clojure.core/println (json/write-str (serialize-curve-trace (nth is-traces-linear %)))) (range 5))

{"inlier-noise":1.2328864339999641,"outlier-noise":0.8763246466225887,"prob-outlier":0.004322114802466692,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[0.8256072193729747,1.8924129325634342]}
{"inlier-noise":0.8259307467200308,"outlier-noise":2.582988859770422,"prob-outlier":0.03817515365794588,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[-0.11003645664821132,1.7903499769271343]}
{"inlier-noise":1.7338166753085809,"outlier-noise":0.7114998505514512,"prob-outlier":0.04826978395551673,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[0.3288061719351629,1.4664164657870762]}
{"inlier-noise":0.9007457873013522,"outlier-noise":1.1714230845947342,"prob-outlier":0.11930833921183503,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[-0.19345049593502733,2.33424732042

(nil nil nil nil nil)

In [35]:
(clojure.core/println (json/write-str (map #(map serialize-curve-trace %) [is-traces-linear])))

[[{"inlier-noise":1.2328864339999641,"outlier-noise":0.8763246466225887,"prob-outlier":0.004322114802466692,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[0.8256072193729747,1.8924129325634342]},{"inlier-noise":0.8259307467200308,"outlier-noise":2.582988859770422,"prob-outlier":0.03817515365794588,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[-0.11003645664821132,1.7903499769271343]},{"inlier-noise":1.7338166753085809,"outlier-noise":0.7114998505514512,"prob-outlier":0.04826978395551673,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[0.3288061719351629,1.4664164657870762]},{"inlier-noise":0.9007457873013522,"outlier-noise":1.1714230845947342,"prob-outlier":0.11930833921183503,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[-0.19345049593502733,2.334247320

In [58]:
(define is-traces-lin-outlier (get-is-traces ys-linear-outlier))
(map #(clojure.core/println (json/write-str (serialize-curve-trace (nth is-traces-lin-outlier %)))) (range 5))

{"inlier-noise":5.906454652069317,"outlier-noise":0.712205456480841,"prob-outlier":0.0158255059329635,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[1.2854194922045061,0.04602980510977737]}
{"inlier-noise":1.0730945256168711,"outlier-noise":6.142978237545821,"prob-outlier":0.1633109164854183,"outlier-designations":[false,true,false,false,true,false,false,false,true,false],"degree":1,"coeffs":[-0.9821520656268071,1.6690942654117684]}
{"inlier-noise":4.662485550994007,"outlier-noise":4.716606052526102,"prob-outlier":0.13008712528227306,"outlier-designations":[false,false,false,false,false,false,true,false,false,false],"degree":1,"coeffs":[1.6142440541481597,0.7027001588527285]}
{"inlier-noise":7.277446544726212,"outlier-noise":0.5575372907547516,"prob-outlier":0.020418461183009273,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":0,"coeffs":[-0.05642961014310482]}
{"inlier-noise":7.4084

(nil nil nil nil nil nil nil nil nil nil)

In [59]:
(define is-traces-quad (get-is-traces ys-quadr))
(map #(clojure.core/println (json/write-str (serialize-curve-trace (nth is-traces-quad %)))) (range 10))

{"inlier-noise":1.5770470821715588,"outlier-noise":0.19640645772834675,"prob-outlier":0.02336277774011264,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":2,"coeffs":[0.07620518799502853,0.27192354102060273,0.2544847324566704]}
{"inlier-noise":1.7770986570722915,"outlier-noise":0.17702593395037386,"prob-outlier":0.03890364264353252,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":2,"coeffs":[-0.1929576612602015,0.11622029759976635,0.21896035659379545]}
{"inlier-noise":1.0409403274210045,"outlier-noise":1.4199784277099754,"prob-outlier":0.08885558275754102,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":2,"coeffs":[-0.043252666721489905,0.3155770519535016,0.29446646490300316]}
{"inlier-noise":1.8441492007921196,"outlier-noise":2.2269159704475716,"prob-outlier":0.033893569814495095,"outlier-designations":[false,false,false,true,false,false,false,false,fa

(nil nil nil nil nil nil nil nil nil nil)

In [60]:
(define is-traces-bump (get-is-traces ys-bump))
(map #(clojure.core/println (json/write-str (serialize-curve-trace (nth is-traces-bump %)))) (range 10))

{"inlier-noise":4.896400328415132,"outlier-noise":1.3729627180561668,"prob-outlier":0.20938051985916817,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":0,"coeffs":[0.18170899594650278]}
{"inlier-noise":5.261315072961767,"outlier-noise":0.8364464650466379,"prob-outlier":0.01902160725850599,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":0,"coeffs":[-0.08400272122905096]}
{"inlier-noise":4.414418521940814,"outlier-noise":3.7927133818227707,"prob-outlier":0.0862871996822645,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[-1.1240029711425128,-0.29287755190489434]}
{"inlier-noise":3.3369196022421037,"outlier-noise":1.4186238671422153,"prob-outlier":0.05210648411971064,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[-3.286663105629336,0.3072639190141064]}
{"inlier-noise":5.72314791559670

(nil nil nil nil nil nil nil nil nil nil)

In [33]:
ys-quadr

(5.5 2.7592592592592586 0.7592592592592595 -0.4999999999999999 2.9814814814814814 -0.7962962962962963 0.16666666666666707 1.8703703703703711 4.314814814814814 7.5)

In [34]:
(clojure.core/println (json/write-str
                          [ys-linear ys-linear-outlier ys-quadr ys-bump]))

CompilerException java.lang.RuntimeException: No such namespace: json, compiling:(/private/var/folders/s4/_p8gnd791cj6nzbd6x5jl34h0000gn/T/form-init1850078609419615396.clj:1:23) 


class clojure.lang.Compiler$CompilerException: 

In [242]:
(json/write-str xs)

"[-5,-3.888888888888889,-2.777777777777778,-1.666666666666667,-0.5555555555555556,0.5555555555555556,1.666666666666667,2.777777777777778,3.888888888888889,5]"

In [241]:
(clojure.core/println (json/write-str (map #(map serialize-curve-trace %) [is-traces-linear is-traces-lin-outlier is-traces-quad is-traces-bump])))

[[{"inlier-noise":0.7285893967575137,"outlier-noise":0.3648463566233224,"prob-outlier":0.06827189146448642,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[0.1598462410362295,2.0112541869639964]},{"inlier-noise":0.6635426445559577,"outlier-noise":0.12749415127731634,"prob-outlier":3.36150611020436E-4,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[-0.5810337117770501,2.0021474107132775]},{"inlier-noise":0.9297229134445932,"outlier-noise":1.275574975309256,"prob-outlier":0.08099026920160479,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[0.6575182098162803,1.9749148235745144]},{"inlier-noise":0.6800911969221412,"outlier-noise":0.36203039581011187,"prob-outlier":0.14010051067848717,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[-0.471617043043263,2.09311690353

In [ ]:
;; Then we need to convert to JSON, and send to a visualizer. Or use Excel...

In [208]:
(define predicted-ys-linear (map (gen [tr] (map (func-from-tr tr) xs)) is-traces-linear))
(clojure.core/print (clojure.string/join "\n" (map str (nth predicted-ys-linear 7))))

-9.210532749724383
-6.940250154615992
-4.6699675595076
-2.399684964399208
-0.12940236929081506
2.140880225817577
4.41116282092597
6.681445416034362
8.951728011142754
11.222010606251146

In [221]:
(map #(trace-value % '("underlying-curve" "coeffs" 2)) is-traces-linear)

(1.872239567838701 -2.0248169956445983 -0.07717290186639941 -0.6870747521532147 -1.8034503381869145 0.1788721315775965 -1.5920565226570949 0.9793296398086003 -0.14892020293232092 0.693934870273943)

{"inlier-noise":0.39428626185130206,"outlier-noise":0.6313633001499879,"prob-outlier":0.020064774114333495,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":2,"coeffs":[-0.45781125522408334,1.9881271076977678,1.872239567838701]}
{"inlier-noise":1.394151204686719,"outlier-noise":2.7760817183108837,"prob-outlier":0.0018087652873206572,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":2,"coeffs":[0.524209534832714,2.2194077982810025,-2.0248169956445983]}
{"inlier-noise":0.8038476264369395,"outlier-noise":0.7502254239988398,"prob-outlier":0.025800316553683866,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":2,"coeffs":[0.488066569859137,2.0785697231143274,-0.07717290186639941]}
{"inlier-noise":0.830718661291917,"outlier-noise":0.565786014229303,"prob-outlier":0.020222092060658725,"outlier-designations":[false,false,false,false,false,false,false,false,false,fa

(nil nil nil nil nil nil nil nil nil nil)

In [ ]:
;; Need to save the traces.
(define export-traces
  (gen [traces]
    (clojure.string/join "\n"
      )))

MCMC algorithm

In [ ]:
(define resimulate-mh
  (gen [model inputs tr addresses]
    (define [current-choices other-choices] (partition-trace tr addresses))
    (define [_ _ current-score] (infer :procedure model :inputs inputs :target-trace other-choices :intervention-trace current-choices))
    (define [_ proposed new-score] (infer :procedure model :inputs inputs :target-trace other-choices))
    (if (flip (exp (- new-score current-score)))
        proposed
        tr)))

In [18]:
(define initial-trace (nth (infer :procedure curve-model :inputs [xs] :target-trace (make-observation-trace ys-linear-outlier)) 1))
(define all-addresses (addresses-of initial-trace))

#'curve-fitting/all-addresses

In [60]:
(def address-contains? 
  (gen [addr elem]
    (some #{elem} addr)))

#'curve-fitting/address-contains?

In [64]:
(def resimulation-mh-move
  (gen [model inputs tr addresses]
    (let [[current-choices fixed-choices]
          (partition-trace tr addresses)

          ;; Get the log probability of the current trace
          [_ _ old-p]
          (infer-and-score :procedure model,
                           :inputs inputs,
                           :observation-trace tr)

          ;; Propose a new trace, and get its score
          [_ proposed new-p-over-forward-q]
          (infer-and-score :procedure model,
                           :inputs inputs,
                           :observation-trace fixed-choices)

          ;; Figure out what the reverse problem would look like
          [_ reverse-move-starting-point]
          (partition-trace proposed addresses)

          ;; Compute a reverse score
          [_ _ reverse-q]
          (infer-and-score :procedure infer-and-score,
                           :inputs [:procedure model,
                                    :inputs inputs,
                                    :observation-trace reverse-move-starting-point]
                           :observation-trace current-choices)

          log-ratio
          (+ new-p-over-forward-q (- reverse-q old-p))]
      (if (flip (exp log-ratio))
        proposed
        tr))))

#'curve-fitting/resimulation-mh-move

In [65]:
(def gaussian-drift-move
  (gen [model inputs tr addresses width]
    (let [proposed-trace
          (reduce 
            (gen [t addr]
              (trace-set-value t addr (+ (trace-value t addr) (* width (gaussian 0 1)))))
              tr addresses)
          
          [_ _ current-score]
          (infer-and-score :procedure model, :inputs inputs, :observation-trace tr)
          
          [_ _ new-score]
          (infer-and-score :procedure model, :inputs inputs, :observation-trace proposed-trace)]

      (if (flip (exp (- new-score current-score)))
        proposed-trace
        tr))))

#'curve-fitting/gaussian-drift-move

In [66]:
(def gibbs-move
  (gen [model inputs tr addresses support]
    (reduce
      (gen [tr addr]
        (let [relative-scores
              (map (gen [choice] 
                     (nth (infer-and-score :procedure model, 
                                           :inputs inputs, 
                                           :observation-trace (trace-set-value tr addr choice)) 
                          2))
                   support)]
          (trace-set-value tr addr (nth support (log-categorical relative-scores)))))
    tr addresses)))

#'curve-fitting/gibbs-move

In [109]:
(def curve-fitting-move
  (gen [tr]
    (let [resimulate-curve
          (gen [tr]
            (resimulation-mh-move 
               curve-model [xs] tr
               (filter #(address-contains? % "underlying-curve") (addresses-of tr))))
          
          resimulate-noise-params
          (gen [tr]
            (resimulation-mh-move
                curve-model [xs] tr
                (filter #(address-contains? % "noise-params") (addresses-of tr))))

          adjust-coeffs
          (gen [tr]
            (gaussian-drift-move 
                curve-model [xs] tr
                (filter #(address-contains? % "coeffs") (addresses-of tr)) 0.1))
          
          adjust-outliers
          (gen [tr]
            (gibbs-move curve-model [xs] tr
                (filter #(address-contains? % "outlier?") (addresses-of tr))
                [true false]))]
     (adjust-outliers (adjust-coeffs (adjust-outliers (adjust-coeffs (adjust-outliers (resimulate-noise-params (adjust-outliers (resimulate-curve tr)))))))))))

#'curve-fitting/curve-fitting-move

In [47]:
(define next-trace
  (gen [trace]
    (define resim-move
     (gen [tr resim-addrs]
       (define [current-choices other-choices] (partition-trace tr resim-addrs))
       (define [_ _ current-score] (infer :procedure curve-model :inputs [xs] :target-trace other-choices :intervention-trace current-choices))
       (pprint current-score)
       (define [_ proposed new-score] (infer :procedure curve-model :inputs [xs] :target-trace other-choices))
       (if (flip (exp (- new-score current-score))) (block (pprint ["RESIM ACCEPTED" new-score (trace-value proposed '("underlying-curve" "degree"))]) (gibbs-outliers-sweep proposed)) tr)))
    
    (define resim-curve
      (gen [tr] (resim-move tr (filter #(address-contains? % "underlying-curve") (addresses-of tr)))))
    
    (define resim-hypers
      (gen [tr] (resim-move tr (filter #(address-contains? % "noise-params") (addresses-of tr)))))
    
    (define walk-coeffs-individually
      (gen [tr] 
        (define [_ _ current-score] (infer :procedure curve-model :target-trace tr :inputs [xs]))
        (define width 0.4)
        (define [final-trace _]
          (reduce
            (gen [[tr scr] next-addr]
              (define proposed-value (+ (trace-value tr next-addr) (* width (gaussian 0 1))))
              (define proposed-trace (trace-set-value tr next-addr proposed-value))
              (define [_ _ new-score] (infer :procedure curve-model :target-trace proposed-trace :inputs [xs]))
              (if (flip (exp (- new-score scr)))
                 [proposed-trace new-score]
                 [tr scr]))
              [tr current-score]
              (filter #(address-contains? % "coeffs") (addresses-of tr))))
         final-trace))
       
    (define walk-coeffs
      (gen [tr]
        (define [_ _ current-score] (infer :procedure curve-model :target-trace tr :inputs [xs]))
        (define width 0.4)
        (define proposed-trace
          (reduce (gen [t addr] (trace-set-value t addr (+ (trace-value t addr) (* width (gaussian 0 1)))))
                  tr 
                  (filter #(address-contains? % "coeffs") (addresses-of tr))))
        (define [_ _ new-score] (infer :procedure curve-model :target-trace proposed-trace :inputs [xs]))
        (if (< (log (uniform 0 1)) (- new-score current-score))
            (block (pprint ["ACCEPTED WALK" new-score]) (gibbs-outliers-sweep proposed-trace))
            tr)))
       
    (define gibbs-outliers-sweep
      (gen [tr]
        (define [_ _ current-score] (infer :procedure curve-model :target-trace tr :inputs [xs]))
        (define [final-trace final-score]
          (reduce
            (gen [[tr scr] next-addr]
             (define current-value (trace-value tr next-addr))
             (define proposed-trace (trace-set-value tr next-addr (not current-value)))
             (define [_ _ new-score] (infer :procedure curve-model :target-trace proposed-trace :inputs [xs]))
             (if (= 0 (log-categorical [scr new-score]))
                [tr scr]
                [proposed-trace new-score]))
            [tr current-score]
            (filter #(address-contains? % "is-outlier") (addresses-of tr))))
         (pprint ["SCORE AFTER GIBBS:" final-score])
         final-trace))
     (define walk-and-gibbs walk-coeffs) ;(gen [tr] (gibbs-outliers-sweep (walk-coeffs tr))))
   (walk-and-gibbs (walk-and-gibbs (resim-hypers (resim-curve trace))))))

#'curve-fitting/next-trace

In [91]:
(let [resimulate-curve
          (gen [tr]
            (resimulation-mh-move 
               curve-model [xs] tr
               (filter #(address-contains? % "underlying-curve") (addresses-of tr))))
          
          resimulate-noise-params
          (gen [tr]
            (resimulation-mh-move
                curve-model [xs] tr
                (filter #(address-contains? % "noise-params") (addresses-of tr))))

          adjust-coeffs
          (gen [tr]
            (gaussian-drift-move 
                curve-model [xs] tr
                (filter #(address-contains? % "coeffs") (addresses-of tr)) 0.1))
          
          adjust-outliers
          (gen [tr]
            (gibbs-move curve-model [xs]
                (filter #(address-contains? % "outlier?") (addresses-of tr)) tr
                [true false]))
         
         
         [_ initial-trace _] (infer-and-score :procedure curve-model :inputs [xs] :observation-trace (make-observation-trace ys))]
    (adjust-outliers initial-trace))

{:first ("data" 0 "outlier?"), :rest (("data" 7 "outlier?") ("data" 1 "outlier?") ("data" 4 "outlier?") ("data" 6 "outlier?") ("data" 3 "outlier?") ("data" 2 "outlier?") ("data" 9 "outlier?") ("data" 5 "outlier?") ("data" 8 "outlier?")), ["underlying-curve" {"degree" {:value 3}, "coeffs" {0 {:value 0.5669614811237799}, 1 {:value -2.035511809175807}, 2 {:value 1.4750395857728074}, 3 {:value 1.6655341477805894}}}] {:value false}, ["noise-params" {"inlier-noise" {:value 0.1630722031648234}, "outlier-noise" {:value 10.159063783512188}, "prob-outlier" {:value 0.3043889793498802}}] {:value true}, ["data" {0 {"outlier?" {:value false}, "y" {:value 6.917680537913382}}, 7 {"outlier?" {:value false}, "y" {:value 42.133341987476115}}, 1 {"outlier?" {:value true}, "y" {:value 1.2471115252763705}}, 4 {"outlier?" {:value true}, "y" {:value 9.075857147908598}}, 6 {"outlier?" {:value true}, "y" {:value 26.87165332124028}}, 3 {"outlier?" {:value false}, "y" {:value 0.4145880558985322}}, 2 {"outlier?" {

In [110]:
(def do-inference
  (gen [ys n]
    (let [[_ initial-trace _] (infer-and-score :procedure curve-model :inputs [xs] :observation-trace (make-observation-trace ys))]
      (reduce
        (gen [tr _] (curve-fitting-move tr))
        initial-trace
        (range n)))))

#'curve-fitting/do-inference

In [113]:
(replicate 5 #(nth (infer-and-score :procedure curve-model :inputs [xs] :observation-trace (do-inference ys-bump 200)) 2))

(-36.02798997153789 -35.65518237137123 -33.23168858850736 -32.42895371590147 -47.41812795777949)

In [44]:
(replicate 3 (gen [] (clojure.core/println (json/write-str (serialize-curve-trace (do-inference ys-linear-outlier 200))))))

-77486.26194881691
["RESIM ACCEPTED" -30618.442323801333]
["SCORE AFTER GIBBS:" -15090.33797909417]
-15090.327759459191
["RESIM ACCEPTED" -5588.511178473314]
["SCORE AFTER GIBBS:" -4347.240169942781]
["ACCEPTED WALK" -4265.796326795904]
["SCORE AFTER GIBBS:" -128.91590469232924]
["ACCEPTED WALK" -129.03230980965685]
["SCORE AFTER GIBBS:" -119.94056714252726]
-115.39260966508166
-119.00438090747917
["RESIM ACCEPTED" -55.99181220372231]
["SCORE AFTER GIBBS:" -55.99556900170774]
["ACCEPTED WALK" -56.071891080223196]
["SCORE AFTER GIBBS:" -56.071891080223196]
["ACCEPTED WALK" -55.976739154730886]
["SCORE AFTER GIBBS:" -55.976739154730886]
-51.46341592879248
-55.972982356745455
["ACCEPTED WALK" -55.955617974768245]
["SCORE AFTER GIBBS:" -55.955617974768245]
-51.437288669036285
-55.95186117678281
["ACCEPTED WALK" -57.532654190821006]
["SCORE AFTER GIBBS:" -57.532654190821006]
-52.89988328925725
-57.528897392835574
["ACCEPTED WALK" -58.57784866171142]
["SCORE AFTER GIBBS:" -58.57784866171142]

(nil nil nil)

In [49]:
(replicate 3 (gen [] (clojure.core/println (json/write-str (serialize-curve-trace (do-inference ys-quadr 500))))))

-5052.8573138764095
-5061.053934868482
["RESIM ACCEPTED" -3781.052378792717 2]
["SCORE AFTER GIBBS:" -1991.9405077731965]
-1984.0535336435444
-1991.03222229865
["RESIM ACCEPTED" -150.1041945884215 2]
["SCORE AFTER GIBBS:" -73.1832395592386]
["ACCEPTED WALK" -77.02676870070347]
["SCORE AFTER GIBBS:" -71.17562530353335]
["ACCEPTED WALK" -71.45035253215926]
["SCORE AFTER GIBBS:" -59.83237277211805]
-51.58494825405168
["RESIM ACCEPTED" -48.57402448455828 0]
["SCORE AFTER GIBBS:" -47.72969849650217]
-46.54016843445891
-43.80384963227803
-46.54016843445891
["RESIM ACCEPTED" -39.96560193567231 0]
["SCORE AFTER GIBBS:" -44.65853343136375]
["ACCEPTED WALK" -45.42878606384656]
["SCORE AFTER GIBBS:" -40.74868177183822]
["ACCEPTED WALK" -40.309321764025086]
["SCORE AFTER GIBBS:" -40.885458541023766]
-36.95171073268182
-36.77491088796827
["ACCEPTED WALK" -40.17936983945664]
["SCORE AFTER GIBBS:" -40.38691120719329]
["ACCEPTED WALK" -40.31743331674543]
["SCORE AFTER GIBBS:" -41.152161406291135]
-37.

(nil nil nil)

In [92]:
(define [_ initial-trace _] (infer :procedure curve-model :inputs [xs] :target-trace (make-observation-trace ys-linear)))

#'curve-fitting/initial-trace

In [ ]:
(do-inference ys-linear-outlier 4)

In [98]:
(define mcmc-samples-linear-outlier (replicate 10 (gen [] (do-inference ys-linear-outlier 500))))
(map #(clojure.core/println (json/write-str (serialize-curve-trace (nth mcmc-samples-linear-outlier %)))) (range 10))

-2663.5156481325616
["RESIM ACCEPTED" -329.7722803726909 0]
["SCORE AFTER GIBBS:" -66.05873304092886]
-64.39309800457029
["RESIM ACCEPTED" -56.08885737741627 0]
["SCORE AFTER GIBBS:" -56.70753676596618]
["ACCEPTED WALK" -56.77057228357465]
["SCORE AFTER GIBBS:" -59.67538444339126]
["ACCEPTED WALK" -59.76020987593125]
["SCORE AFTER GIBBS:" -56.74186857716094]
-52.74232668515906
-56.123189188611036
["ACCEPTED WALK" -57.8004272846524]
["SCORE AFTER GIBBS:" -57.8004272846524]
["ACCEPTED WALK" -57.449230066574714]
["SCORE AFTER GIBBS:" -57.449230066574714]
-53.48421201143698
-56.83055067802481
["ACCEPTED WALK" -57.006390473701465]
["SCORE AFTER GIBBS:" -57.006390473701465]
-53.02628047265209
-56.38771108515156
["RESIM ACCEPTED" -53.046022994474214 0]
["SCORE AFTER GIBBS:" -58.474019980476555]
["ACCEPTED WALK" -58.72555932730467]
["SCORE AFTER GIBBS:" -55.5778308362494]
["ACCEPTED WALK" -55.349921524452526]
["SCORE AFTER GIBBS:" -55.45186807878282]
-51.51370198900467
["RESIM ACCEPTED" -52.83

(nil nil nil nil nil nil nil nil nil nil)

In [94]:
(define mcmc-samples-linear (replicate 10 (gen [] (do-inference ys-linear 500))))
(map #(clojure.core/println (json/write-str (serialize-curve-trace (nth mcmc-samples-linear %)))) (range 10))

-314796.1212012844
-314803.04664010217
["ACCEPTED WALK" -211047.28073851697]
["SCORE AFTER GIBBS:" -165997.04914622664]
-165990.24455242656
["RESIM ACCEPTED" -5254.846625263525 3]
["SCORE AFTER GIBBS:" -44.26323749966581]
-44.27481343380614
-37.55294238650938
-44.27481343380614
-37.55294238650938
-44.27481343380614
-37.55294238650938
-44.27481343380614
-37.55294238650938
-44.27481343380614
["RESIM ACCEPTED" -44.36056100652495 3]
["SCORE AFTER GIBBS:" -44.35899079894592]
-37.648695685789484
-44.36056100652495
["RESIM ACCEPTED" -42.83141433188993 3]
["SCORE AFTER GIBBS:" -44.63180879316923]
-37.92151368001279
-40.00080583362598
-37.92151368001279
-40.00080583362598
-37.92151368001279
-40.00080583362598
-37.92151368001279
-40.00080583362598
-37.92151368001279
-40.00080583362598
-37.92151368001279
-40.00080583362598
-37.92151368001279
-40.00080583362598
-37.92151368001279
-40.00080583362598
-37.92151368001279
-40.00080583362598
-37.92151368001279
-40.00080583362598
-37.92151368001279
-40.0

(nil nil nil nil nil nil nil nil nil nil)

In [50]:
(define mcmc-samples-quadr (replicate 10 (gen [] (do-inference ys-quadr 500))))
(map #(clojure.core/println (json/write-str (serialize-curve-trace (nth mcmc-samples-quadr %)))) (range 10))

-45.9428109897009
-47.22735812108272
["ACCEPTED WALK" -43.70956771871468]
["SCORE AFTER GIBBS:" -42.49254074661194]
["ACCEPTED WALK" -42.22778691944674]
["SCORE AFTER GIBBS:" -42.56129772898949]
-38.017719579165586
-39.072703090345385
["ACCEPTED WALK" -42.37099442777842]
["SCORE AFTER GIBBS:" -45.07175560713745]
["ACCEPTED WALK" -43.976435864391824]
["SCORE AFTER GIBBS:" -43.329156325020456]
-38.865649909972625
-39.84056168637636
["ACCEPTED WALK" -43.27191909592978]
["SCORE AFTER GIBBS:" -42.46455021867361]
-38.003282814152506
-38.975955580029506
["ACCEPTED WALK" -41.6532205025759]
["SCORE AFTER GIBBS:" -41.129519840350746]
-36.71300393948305
-37.64092520170665
-36.71300393948305
-37.64092520170665
["ACCEPTED WALK" -41.02883057341947]
["SCORE AFTER GIBBS:" -42.37446195246475]
["ACCEPTED WALK" -41.70444118829493]
["SCORE AFTER GIBBS:" -38.292259612558446]
-33.92226910019508
-34.803664973914344
["ACCEPTED WALK" -38.991088296838]
["SCORE AFTER GIBBS:" -43.43887923617558]
-39.0429403082601

(nil nil nil nil nil nil nil nil nil nil)

In [51]:
(clojure.core/println (json/write-str(map #(serialize-curve-trace (nth mcmc-samples-quadr %)) (range 10))))

[{"inlier-noise":1.304952004576469,"outlier-noise":8.367044130496362,"prob-outlier":0.1254289277002256,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":3,"coeffs":[0.9615825593469994,-0.2436614294394242,0.2610712962134925,0.01900466164808343]},{"inlier-noise":1.3753125458174218,"outlier-noise":7.244451408593593,"prob-outlier":0.06793107992092472,"outlier-designations":[false,false,false,false,true,false,false,false,false,false],"degree":2,"coeffs":[-0.4707770222517963,0.06565660502067172,0.23218301775783834]},{"inlier-noise":0.2610404319591143,"outlier-noise":6.851804665113205,"prob-outlier":0.11983367373503546,"outlier-designations":[false,false,false,false,true,false,false,false,false,false],"degree":2,"coeffs":[-1.0035053567229861,0.25814171386257956,0.29795519875996146]},{"inlier-noise":0.7261617635093727,"outlier-noise":10.548953576494473,"prob-outlier":0.08903333361835185,"outlier-designations":[false,false,false,false,true,false,false

In [53]:
(clojure.core/println (json/write-str(map #(serialize-curve-trace (nth mcmc-samples-bump %)) (range 10))))

[{"inlier-noise":1.306865672864964,"outlier-noise":5.233034695114949,"prob-outlier":0.06643863384283429,"outlier-designations":[false,false,false,false,false,false,false,true,false,false],"degree":0,"coeffs":[-5.912624453924092]},{"inlier-noise":1.2763735443406632,"outlier-noise":6.309933915643409,"prob-outlier":0.07567589322197729,"outlier-designations":[false,false,false,false,false,false,false,true,false,false],"degree":1,"coeffs":[-5.936573328523336,-0.5066232828961479]},{"inlier-noise":1.4793280007677947,"outlier-noise":5.382873300626187,"prob-outlier":0.12293941130084998,"outlier-designations":[false,false,false,false,false,false,false,true,false,false],"degree":1,"coeffs":[-5.944137987229647,-0.6821098085328197]},{"inlier-noise":1.8718691901140023,"outlier-noise":11.63714953881994,"prob-outlier":0.08146107286373906,"outlier-designations":[false,false,false,false,false,false,false,true,false,false],"degree":0,"coeffs":[-6.998708047969252]},{"inlier-noise":0.7021749264855346,"outl

In [52]:
(define mcmc-samples-bump (replicate 10 (gen [] (do-inference ys-bump 500))))
(map #(clojure.core/println (json/write-str (serialize-curve-trace (nth mcmc-samples-bump %)))) (range 10))

-508.0839752129529
-512.0061737711826
["RESIM ACCEPTED" -161.36263973159296 0]
["SCORE AFTER GIBBS:" -41.53157160140386]
-36.60952461522015
-39.792158590659106
["ACCEPTED WALK" -40.33897329372553]
["SCORE AFTER GIBBS:" -41.46738848383682]
-36.59198636063629
-39.727975473092066
-36.59198636063629
-39.727975473092066
["ACCEPTED WALK" -41.54907500088579]
["SCORE AFTER GIBBS:" -40.86079029316099]
-35.96406974249417
-39.12137728241623
["ACCEPTED WALK" -41.23545045571133]
["SCORE AFTER GIBBS:" -41.429158105505294]
-36.51804048747404
-39.68974509476054
["ACCEPTED WALK" -40.83470269517365]
["SCORE AFTER GIBBS:" -40.83470269517365]
["ACCEPTED WALK" -40.076700430203886]
["SCORE AFTER GIBBS:" -40.076700430203886]
-35.287394933235355
-38.33728741945913
["ACCEPTED WALK" -40.21276452407088]
["SCORE AFTER GIBBS:" -37.607211655841695]
["ACCEPTED WALK" -37.87107347412021]
["SCORE AFTER GIBBS:" -37.87107347412021]
-33.275844850621
-36.131660463375454
["ACCEPTED WALK" -37.91862026701734]
["SCORE AFTER GI

(nil nil nil nil nil nil nil nil nil nil)

In [291]:
(apply + (replicate 10 (gen [] (gamma 2 1))))

20.224488402639317

In [69]:
(define importance-sampling-results
'(({:inlier-noise 0.3203025826287354, :outlier-noise 2.1827745078053087, :prob-outlier 0.10346420471651874, :outlier-designations (false false false false false false false false false false), :degree 1, :coeffs (0.25942556137914496 2.080606069842025)} {:inlier-noise 0.15104982569100756, :outlier-noise 0.7195928851383143, :prob-outlier 0.02744888382649424, :outlier-designations (false false false false false false false false false false), :degree 1, :coeffs (-0.04140218418946105 1.9978719710057904)} {:inlier-noise 0.2934489489359625, :outlier-noise 0.9977242271034761, :prob-outlier 8.696072799625476E-4, :outlier-designations (false false false false false false false false false false), :degree 1, :coeffs (-0.15748523674217008 1.8834863244894027)} {:inlier-noise 0.30115351907696647, :outlier-noise 3.6001745466642445, :prob-outlier 0.1657168472360676, :outlier-designations (false false false false false false false false false false), :degree 1, :coeffs (0.18134993768205582 2.0288427396849062)} {:inlier-noise 0.28712174500409615, :outlier-noise 1.383204786902018, :prob-outlier 0.1936035309265709, :outlier-designations (false false false false false true true false false false), :degree 1, :coeffs (-0.24802180725626324 1.988229672095906)} {:inlier-noise 0.8046156063227546, :outlier-noise 0.634261238065272, :prob-outlier 0.01293693904046922, :outlier-designations (false false false false false false false false false false), :degree 1, :coeffs (0.42952043743688717 1.969625108103368)} {:inlier-noise 0.6048009521213341, :outlier-noise 0.7041344683269578, :prob-outlier 0.021726011562084313, :outlier-designations (false false false false false false false false false false), :degree 1, :coeffs (-0.07689009832033919 2.077300305142018)} {:inlier-noise 0.5132178283388329, :outlier-noise 1.1469921224563022, :prob-outlier 0.05008700283523149, :outlier-designations (false false false false true false false false false false), :degree 1, :coeffs (0.36091272465001745 1.8893606624263402)} {:inlier-noise 0.6589377728434536, :outlier-noise 1.7709760153044656, :prob-outlier 0.05793699675883146, :outlier-designations (false false false false false false false false false false), :degree 1, :coeffs (-0.2454549097821225 2.006830312196887)} {:inlier-noise 0.8019957699607595, :outlier-noise 4.279985544922286, :prob-outlier 0.01850101222457079, :outlier-designations (false false false false false false false false false false), :degree 1, :coeffs (-0.018658869084247107 1.932774515279486)}) ({:inlier-noise 5.906454652069317, :outlier-noise 0.712205456480841, :prob-outlier 0.0158255059329635, :outlier-designations (false false false false false false false false false false), :degree 1, :coeffs (1.2854194922045061 0.04602980510977737)} {:inlier-noise 1.0730945256168711, :outlier-noise 6.142978237545821, :prob-outlier 0.1633109164854183, :outlier-designations (false true false false true false false false true false), :degree 1, :coeffs (-0.9821520656268071 1.6690942654117684)} {:inlier-noise 4.662485550994007, :outlier-noise 4.716606052526102, :prob-outlier 0.13008712528227306, :outlier-designations (false false false false false false true false false false), :degree 1, :coeffs (1.6142440541481597 0.7027001588527285)} {:inlier-noise 7.277446544726212, :outlier-noise 0.5575372907547516, :prob-outlier 0.020418461183009273, :outlier-designations (false false false false false false false false false false), :degree 0, :coeffs (-0.05642961014310482)} {:inlier-noise 7.408414026353044, :outlier-noise 3.5430037966681214, :prob-outlier 0.043187626459242456, :outlier-designations (false false false false false false false false false false), :degree 0, :coeffs (1.3333514434464429)} {:inlier-noise 0.31219850191517085, :outlier-noise 3.338906072744802, :prob-outlier 0.09968095018392398, :outlier-designations (false true false false false false false false true false), :degree 1, :coeffs (0.08942101868203549 1.9611536774361777)} {:inlier-noise 6.055315521893033, :outlier-noise 3.431815548211557, :prob-outlier 0.14145747939271902, :outlier-designations (false false false false false true true false false false), :degree 0, :coeffs (0.011841535792496053)} {:inlier-noise 5.1285379585511555, :outlier-noise 1.0970094501898071, :prob-outlier 0.14738578320715623, :outlier-designations (false false false false false false false false false false), :degree 1, :coeffs (-0.5059636910140588 0.7310564370817128)} {:inlier-noise 6.335800326083608, :outlier-noise 1.5353779846738818, :prob-outlier 0.08383226996151028, :outlier-designations (false false false false false true false false false false), :degree 0, :coeffs (-1.3985874474719757)} {:inlier-noise 6.329675616015294, :outlier-noise 1.6603426559941301, :prob-outlier 0.2850667830064594, :outlier-designations (false false false false false true false false false false), :degree 0, :coeffs (0.2893373080219742)}) ({:inlier-noise 1.5770470821715588, :outlier-noise 0.19640645772834675, :prob-outlier 0.02336277774011264, :outlier-designations (false false false false false false false false false false), :degree 2, :coeffs (0.07620518799502853 0.27192354102060273 0.2544847324566704)} {:inlier-noise 1.7770986570722915, :outlier-noise 0.17702593395037386, :prob-outlier 0.03890364264353252, :outlier-designations (false false false false false false false false false false), :degree 2, :coeffs (-0.1929576612602015 0.11622029759976635 0.21896035659379545)} {:inlier-noise 1.0409403274210045, :outlier-noise 1.4199784277099754, :prob-outlier 0.08885558275754102, :outlier-designations (false false false false false false false false false false), :degree 2, :coeffs (-0.043252666721489905 0.3155770519535016 0.29446646490300316)} {:inlier-noise 1.8441492007921196, :outlier-noise 2.2269159704475716, :prob-outlier 0.033893569814495095, :outlier-designations (false false false true false false false false false false), :degree 2, :coeffs (0.1618013605544861 0.11284372402932998 0.23334018655756408)} {:inlier-noise 2.658333709964433, :outlier-noise 0.9020931385673682, :prob-outlier 0.014071800882081265, :outlier-designations (false false false false false false false false false false), :degree 2, :coeffs (0.1344196167121793 0.1034715374620226 0.31057120974657376)} {:inlier-noise 1.18309346993312, :outlier-noise 5.1379754053415105, :prob-outlier 0.02124600339610172, :outlier-designations (false false false false false false false false false false), :degree 2, :coeffs (0.8463302606408691 0.24207461633636074 0.2631270129767408)} {:inlier-noise 1.8027812835628514, :outlier-noise 0.6113527903816398, :prob-outlier 0.003073978362274965, :outlier-designations (false false false false false false false false false false), :degree 2, :coeffs (-0.4615514704098335 -0.013617258939551104 0.26200202045311244)} {:inlier-noise 1.382212881892258, :outlier-noise 3.676939545544927, :prob-outlier 0.01252129352875786, :outlier-designations (false false false false false false false false false false), :degree 2, :coeffs (-0.24247197956946354 0.1821376084055264 0.28102008624688524)} {:inlier-noise 1.2221772169187735, :outlier-noise 1.8963203655017824, :prob-outlier 0.056521618603058266, :outlier-designations (false false false false false false false false false false), :degree 2, :coeffs (0.35809158305170424 -0.1441347211803391 0.20408045682388315)} {:inlier-noise 1.4273505070009542, :outlier-noise 0.9632976521078445, :prob-outlier 0.11910002323416413, :outlier-designations (false false false false false false false false false false), :degree 2, :coeffs (0.33951458416545693 0.3286293768001541 0.24290582831152496)}) ({:inlier-noise 4.896400328415132, :outlier-noise 1.3729627180561668, :prob-outlier 0.20938051985916817, :outlier-designations (false false false false false false false false false false), :degree 0, :coeffs (0.18170899594650278)} {:inlier-noise 5.261315072961767, :outlier-noise 0.8364464650466379, :prob-outlier 0.01902160725850599, :outlier-designations (false false false false false false false false false false), :degree 0, :coeffs (-0.08400272122905096)} {:inlier-noise 4.414418521940814, :outlier-noise 3.7927133818227707, :prob-outlier 0.0862871996822645, :outlier-designations (false false false false false false false false false false), :degree 1, :coeffs (-1.1240029711425128 -0.29287755190489434)} {:inlier-noise 3.3369196022421037, :outlier-noise 1.4186238671422153, :prob-outlier 0.05210648411971064, :outlier-designations (false false false false false false false false false false), :degree 1, :coeffs (-3.286663105629336 0.3072639190141064)} {:inlier-noise 5.723147915596702, :outlier-noise 0.833309134199272, :prob-outlier 0.0630311305592377, :outlier-designations (false false false false false false false false false false), :degree 0, :coeffs (-0.6338365870166326)} {:inlier-noise 3.6974130416787965, :outlier-noise 2.314195477500819, :prob-outlier 0.038822668078874134, :outlier-designations (false false false false false false false false false false), :degree 0, :coeffs (-1.3489157053958267)} {:inlier-noise 4.1388794230251325, :outlier-noise 1.5706347441266022, :prob-outlier 7.224710141837676E-4, :outlier-designations (false false false false false false false false false false), :degree 1, :coeffs (-1.9011786269925302 -0.3120371490504696)} {:inlier-noise 5.922435232278974, :outlier-noise 0.9258877142160893, :prob-outlier 0.047111631046466096, :outlier-designations (false false false false false false false false false false), :degree 2, :coeffs (-0.1334759245625091 -0.6067026326348669 -0.4658458819308759)} {:inlier-noise 3.850814319744413, :outlier-noise 0.8766524630453508, :prob-outlier 0.022733072175657676, :outlier-designations (false false false false false false false false false false), :degree 2, :coeffs (-1.4376963007825947 -0.19461010706927698 -0.2148990550268955)} {:inlier-noise 5.817549483535617, :outlier-noise 1.3823738166211195, :prob-outlier 8.57477760971781E-4, :outlier-designations (false false false false false false false false false false), :degree 0, :coeffs (-1.8600067750358364)})))

#'curve-fitting/importance-sampling-results

In [73]:
(define mcmc-results '(({:inlier-noise 0.013709773427963446, :outlier-noise 0.3338742920353115, :prob-outlier 0.008169187361124597, :outlier-designations (false false false false false false false false false false), :degree 2, :coeffs (0.00844442912323969 2.0021147942411335 -7.789214645188821E-4)} {:inlier-noise 0.004472938492709234, :outlier-noise 0.2845577455385088, :prob-outlier 0.17044019775894736, :outlier-designations (false false false false false false false false false false), :degree 1, :coeffs (0.0017776681286905708 1.9997148354002485)} {:inlier-noise 0.004749865723390128, :outlier-noise 1.8796268703038255, :prob-outlier 0.003795485058423198, :outlier-designations (false false false false false false false false false false), :degree 1, :coeffs (9.530771714335341E-4 2.0005547489320414)} {:inlier-noise 0.0164677897400492, :outlier-noise 1.1194308915473872, :prob-outlier 0.004211249069291045, :outlier-designations (false false false false false false false false false false), :degree 1, :coeffs (0.00847262050454236 2.0011726947802657)} {:inlier-noise 0.0017644274053868663, :outlier-noise 3.5615605231547773, :prob-outlier 0.03537277836134778, :outlier-designations (false false false false false false false false false false), :degree 1, :coeffs (6.696118587613156E-4 2.0002183339475765)} {:inlier-noise 0.005254043376993837, :outlier-noise 0.2728986948677942, :prob-outlier 0.01692446013570481, :outlier-designations (false false false false false false false false false false), :degree 2, :coeffs (0.0029764644462073357 1.9981682751941396 -1.7317023601081392E-4)} {:inlier-noise 0.005562855506508254, :outlier-noise 1.205800492207462, :prob-outlier 0.15546511423674725, :outlier-designations (false false false false false false false false false false), :degree 2, :coeffs (0.003997990385612481 1.9997755298696522 -3.0825320440697136E-4)} {:inlier-noise 0.013416504680463463, :outlier-noise 1.3552420553308764, :prob-outlier 0.052523537197219805, :outlier-designations (false false false false false false false false false false), :degree 1, :coeffs (0.005766521634235007 1.9998504904342222)} {:inlier-noise 0.0014993023002445357, :outlier-noise 1.3688588910891337, :prob-outlier 0.11141653325197365, :outlier-designations (false false false false false false false false false false), :degree 1, :coeffs (-4.497168771025418E-5 2.0001512468922624)} {:inlier-noise 0.005576758936314168, :outlier-noise 0.6542061806789493, :prob-outlier 0.015542798274123082, :outlier-designations (false false false false false false false false false false), :degree 1, :coeffs (-6.114940400871794E-4 2.0014116527083674)}) ({:inlier-noise 0.018194996479197527, :outlier-noise 6.858966154342858, :prob-outlier 0.09608776041165257, :outlier-designations (false true false false false false false false true false), :degree 1, :coeffs (0.0067164982659762285 1.9975638197153718)} {:inlier-noise 6.055204136193853, :outlier-noise 3.657466504650155, :prob-outlier 0.0576288540723382, :outlier-designations (false false false true false false false false false false), :degree 0, :coeffs (-0.7737357947237932)} {:inlier-noise 0.0872231188867097, :outlier-noise 7.315966130380839, :prob-outlier 0.4264614196621326, :outlier-designations (false true false false false false false false true false), :degree 1, :coeffs (-0.025722051816238826 1.9964679580998057)} {:inlier-noise 7.463412449452533, :outlier-noise 3.885493210441845, :prob-outlier 0.03370989639828226, :outlier-designations (false false false false false false false false false false), :degree 1, :coeffs (-1.5681355605943925 0.10995468549561896)} {:inlier-noise 1.662694697810827, :outlier-noise 5.689134711572143, :prob-outlier 0.1783435646024315, :outlier-designations (true false true false false false false false true false), :degree 3, :coeffs (-0.16425549947469542 1.8624861312309267 0.6563753516963801 -0.14804573880074828)} {:inlier-noise 7.280560452365689, :outlier-noise 0.5512941130656074, :prob-outlier 0.08830940852831703, :outlier-designations (false false false false false false false false false false), :degree 0, :coeffs (1.2516929812932611)} {:inlier-noise 0.14232363391198252, :outlier-noise 7.030185487458045, :prob-outlier 0.09416902656197268, :outlier-designations (false true false false false false false false true false), :degree 1, :coeffs (-0.02665715388494181 1.9795319120462644)} {:inlier-noise 3.3974714638611117, :outlier-noise 3.580941185076756, :prob-outlier 0.17377825012842485, :outlier-designations (true false false false false false false false false true), :degree 3, :coeffs (-1.006665753939872 1.4938335720961768 0.12178372298365621 -0.26160000890019036)} {:inlier-noise 0.06229805073709528, :outlier-noise 6.616975686080237, :prob-outlier 0.35799221732398023, :outlier-designations (false true false false false false false false true false), :degree 1, :coeffs (-0.028171878835608163 2.0088534637094626)} {:inlier-noise 5.438683719567188, :outlier-noise 2.53453139225407, :prob-outlier 0.11519594111080578, :outlier-designations (false false false false false false false false false false), :degree 0, :coeffs (0.13611419079678438)}) ({:inlier-noise 0.0461955835310787, :outlier-noise 1.721532372749385, :prob-outlier 0.1638044336174317, :outlier-designations (false false false false true false false false false false), :degree 2, :coeffs (-0.9876401968845321 0.19796732511866752 0.2996484349759008)} {:inlier-noise 0.014649983590320262, :outlier-noise 1.9545270154050631, :prob-outlier 0.06948351564411037, :outlier-designations (false false false false true false false false false false), :degree 2, :coeffs (-1.0002413046249965 0.20268963503509219 0.29941280684882554)} {:inlier-noise 0.02831524269239194, :outlier-noise 1.7406723981824506, :prob-outlier 0.05628920872707843, :outlier-designations (false false false false true false false false false false), :degree 2, :coeffs (-0.9487206076262601 0.1979762972997453 0.29809759624450916)} {:inlier-noise 0.06356847829536352, :outlier-noise 2.820276795921456, :prob-outlier 0.20340945684125522, :outlier-designations (false false false false true false false false false false), :degree 2, :coeffs (-1.0419330943159248 0.20030418233247047 0.3036865834889395)} {:inlier-noise 0.008953805322384599, :outlier-noise 3.487778644935057, :prob-outlier 0.049634614066853544, :outlier-designations (false false false false true false false false false false), :degree 2, :coeffs (-1.0042955251351091 0.20045256727803826 0.3001754902576207)} {:inlier-noise 0.05915005747347897, :outlier-noise 3.5420748777477065, :prob-outlier 0.13208922934999212, :outlier-designations (false false false false true false false false false false), :degree 2, :coeffs (-0.9866262180233557 0.20525904407849851 0.29954848342635204)} {:inlier-noise 0.01566985421075981, :outlier-noise 1.6178788000498385, :prob-outlier 0.07971268992838043, :outlier-designations (false false false false true false false false false false), :degree 2, :coeffs (-0.9871488832253534 0.20139054232712617 0.29869336889187664)} {:inlier-noise 0.020629682702384706, :outlier-noise 1.6975880673161996, :prob-outlier 0.09427322277552797, :outlier-designations (false false false false true false false false false false), :degree 2, :coeffs (-0.9710048436410446 0.19953667466883718 0.29841451332289876)} {:inlier-noise 0.009157211222417317, :outlier-noise 2.273905181949468, :prob-outlier 0.09611707326883401, :outlier-designations (false false false false true false false false false false), :degree 2, :coeffs (-1.0053831495835663 0.20048189606140937 0.30031816282075363)} {:inlier-noise 0.2870710759898136, :outlier-noise 2.143441905723575, :prob-outlier 0.14364830537091533, :outlier-designations (false false false false true false false false false false), :degree 3, :coeffs (-0.8316499960662787 0.0453993098036289 0.27844757498934425 0.005533500649518993)}) ({:inlier-noise 0.898795720530503, :outlier-noise 5.4553741344640425, :prob-outlier 0.23695877304039203, :outlier-designations (false false false false true true true false true true), :degree 2, :coeffs (-1.7119896520371125 2.4073402702201037 0.4242612213838156)} {:inlier-noise 6.053408445329012, :outlier-noise 3.4540620327028235, :prob-outlier 0.0629733028349605, :outlier-designations (false false false false false false false false false false), :degree 1, :coeffs (0.3359941714723739 0.13745951497264858)} {:inlier-noise 4.971156231609132, :outlier-noise 0.28076658761126994, :prob-outlier 0.09198859699943884, :outlier-designations (false false false false false false false false false false), :degree 1, :coeffs (-2.2893874314527025 -0.10439481763286107)} {:inlier-noise 4.771158793492894, :outlier-noise 1.8781128899920898, :prob-outlier 0.0061443896575312795, :outlier-designations (false false false false false false false false false false), :degree 0, :coeffs (-2.0318943043913062)} {:inlier-noise 5.088823251025101, :outlier-noise 0.7941416239504905, :prob-outlier 0.051731704284792235, :outlier-designations (false false false false false false false false false false), :degree 0, :coeffs (-0.9454639015543744)} {:inlier-noise 2.4997744645618805, :outlier-noise 3.4890303034435486, :prob-outlier 0.13458186084051615, :outlier-designations (false false false false false false false true false false), :degree 0, :coeffs (-2.9286102105676273)} {:inlier-noise 2.5526862759176194, :outlier-noise 3.8615195018820274, :prob-outlier 0.16828051782332853, :outlier-designations (false false false false false false false true false false), :degree 2, :coeffs (-3.0004048284587004 -0.06027410255069754 -0.13372398181843487)} {:inlier-noise 4.820612174306468, :outlier-noise 1.5617176538265083, :prob-outlier 0.05914039816753991, :outlier-designations (false false false false false false false false false false), :degree 1, :coeffs (-0.8689889767033437 0.28007669615649117)} {:inlier-noise 1.953617483084775, :outlier-noise 4.071684836127407, :prob-outlier 0.03799824177242905, :outlier-designations (false false false false false false false true false false), :degree 0, :coeffs (-3.4337079622146836)} {:inlier-noise 3.0580137332519497, :outlier-noise 4.644741496348044, :prob-outlier 0.24111132757184406, :outlier-designations (false false false true true false false true true false), :degree 2, :coeffs (-1.9075451686702118 0.09546403750947681 -0.10461116256083526)})))

#'curve-fitting/mcmc-results

In [72]:
mcmc-results

(({:inlier-noise 0.013709773427963446, :outlier-noise 0.3338742920353115, :prob-outlier 0.008169187361124597, :outlier-designations (false false false false false false false false false false), :degree 2, :coeffs (0.00844442912323969 2.0021147942411335 -7.789214645188821E-4)} {:inlier-noise 0.004472938492709234, :outlier-noise 0.2845577455385088, :prob-outlier 0.17044019775894736, :outlier-designations (false false false false false false false false false false), :degree 1, :coeffs (0.0017776681286905708 1.9997148354002485)} {:inlier-noise 0.004749865723390128, :outlier-noise 1.8796268703038255, :prob-outlier 0.003795485058423198, :outlier-designations (false false false false false false false false false false), :degree 1, :coeffs (9.530771714335341E-4 2.0005547489320414)} {:inlier-noise 0.0164677897400492, :outlier-noise 1.1194308915473872, :prob-outlier 0.004211249069291045, :outlier-designations (false false false false false false false false false false), :degree 1, :coeffs (0

In [ ]:
(clojure.core/println (json/write-str (map #(map serialize-curve-trace %) [mcmc-samples-linear-outlier])))

In [55]:
(clojure.core/println (json/write-str (map #(map serialize-curve-trace %) [mcmc-samples-linear-outlier])))

[[{"inlier-noise":6.092620345965712,"outlier-noise":2.104120974299457,"prob-outlier":0.11638870309483829,"outlier-designations":[false,false,false,false,false,true,false,false,false,false],"degree":0,"coeffs":[-1.5156452902838347]},{"inlier-noise":1.274346199693879,"outlier-noise":5.647396344033266,"prob-outlier":0.3274058713338496,"outlier-designations":[true,false,true,true,false,false,false,false,true,true],"degree":2,"coeffs":[0.006083070328619156,-0.07799970802236048,0.737305301124805]},{"inlier-noise":6.380241192611475,"outlier-noise":5.441481300784857,"prob-outlier":0.10188118995818018,"outlier-designations":[false,false,false,false,true,true,false,false,false,false],"degree":1,"coeffs":[1.807424296885631,0.24752590855971185]},{"inlier-noise":0.10415602754298173,"outlier-noise":3.5644832352659708,"prob-outlier":0.01653486673528648,"outlier-designations":[false,true,false,false,false,false,false,false,true,false],"degree":1,"coeffs":[-0.009073186337204837,2.0133153228040173]},{"i

In [299]:
(clojure.core/println (json/write-str (map #(map serialize-curve-trace %) [is-traces-linear mcmc-samples-linear-outlier is-traces-quad is-traces-bump])))

[[{"inlier-noise":0.7285893967575137,"outlier-noise":0.3648463566233224,"prob-outlier":0.06827189146448642,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[0.1598462410362295,2.0112541869639964]},{"inlier-noise":0.6635426445559577,"outlier-noise":0.12749415127731634,"prob-outlier":3.36150611020436E-4,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[-0.5810337117770501,2.0021474107132775]},{"inlier-noise":0.9297229134445932,"outlier-noise":1.275574975309256,"prob-outlier":0.08099026920160479,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[0.6575182098162803,1.9749148235745144]},{"inlier-noise":0.6800911969221412,"outlier-noise":0.36203039581011187,"prob-outlier":0.14010051067848717,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[-0.471617043043263,2.09311690353

In [76]:
; from before
(clojure.core/println (json/write-str importance-sampling-results))

[[{"inlier-noise":0.3203025826287354,"outlier-noise":2.1827745078053087,"prob-outlier":0.10346420471651874,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[0.25942556137914496,2.080606069842025]},{"inlier-noise":0.15104982569100756,"outlier-noise":0.7195928851383143,"prob-outlier":0.02744888382649424,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[-0.04140218418946105,1.9978719710057904]},{"inlier-noise":0.2934489489359625,"outlier-noise":0.9977242271034761,"prob-outlier":8.696072799625476E-4,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[-0.15748523674217008,1.8834863244894027]},{"inlier-noise":0.30115351907696647,"outlier-noise":3.6001745466642445,"prob-outlier":0.1657168472360676,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[0.18134993768205582,2.028842

In [89]:
; after lin-outliers
(clojure.core/println (json/write-str (map #(map serialize-curve-trace %) [is-traces-linear is-traces-lin-outlier is-traces-quad is-traces-bump])))

CompilerException java.lang.RuntimeException: Unable to resolve symbol: is-traces-linear in this context, compiling:(/private/var/folders/s4/_p8gnd791cj6nzbd6x5jl34h0000gn/T/form-init2469254994330254184.clj:2:39) 


class clojure.lang.Compiler$CompilerException: 

In [96]:
; after lin-outliers
(clojure.core/println (json/write-str (map #(map serialize-curve-trace %) [is-traces-linear mcmc-samples-linear-outlier is-traces-quad is-traces-bump])))

[[{"inlier-noise":0.3203025826287354,"outlier-noise":2.1827745078053087,"prob-outlier":0.10346420471651874,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[0.25942556137914496,2.080606069842025]},{"inlier-noise":0.15104982569100756,"outlier-noise":0.7195928851383143,"prob-outlier":0.02744888382649424,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[-0.04140218418946105,1.9978719710057904]},{"inlier-noise":0.2934489489359625,"outlier-noise":0.9977242271034761,"prob-outlier":8.696072799625476E-4,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[-0.15748523674217008,1.8834863244894027]},{"inlier-noise":0.30115351907696647,"outlier-noise":3.6001745466642445,"prob-outlier":0.1657168472360676,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[0.18134993768205582,2.028842

In [ ]:
; after quadr
(clojure.core/println (json/write-str (map #(map serialize-curve-trace %) [is-traces-linear mcmc-samples-linear-outlier is-traces-quad is-traces-bump])))

In [ ]:
; after bump
(clojure.core/println (json/write-str (map #(map serialize-curve-trace %) [is-traces-linear mcmc-samples-linear-outlier is-traces-quad is-traces-bump])))

In [100]:
; after all
(clojure.core/println (json/write-str (map #(map serialize-curve-trace %) [mcmc-samples-linear mcmc-samples-linear-outlier mcmc-samples-quadr mcmc-samples-bump])))

[[{"inlier-noise":0.010197105340386365,"outlier-noise":4.142880068793949,"prob-outlier":0.13596613079887462,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[-0.005079624403207431,2.0004934129914216]},{"inlier-noise":0.021834182594597705,"outlier-noise":12.198000360301439,"prob-outlier":0.054360378346622315,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[0.025311034626876887,2.0008564257941894]},{"inlier-noise":0.0643767352077578,"outlier-noise":11.356354030256911,"prob-outlier":0.06220582369052091,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[0.04246815615866904,1.993196027714258]},{"inlier-noise":0.006800564885112691,"outlier-noise":10.642626763918537,"prob-outlier":0.0482429003450712,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[0.00376646494451649,1.99

In [101]:
(clojure.core/println (json/write-str (map #(map serialize-curve-trace %) [mcmc-samples-linear mcmc-samples-linear-outlier mcmc-samples-quadr mcmc-samples-bump])))

[[{"inlier-noise":0.010197105340386365,"outlier-noise":4.142880068793949,"prob-outlier":0.13596613079887462,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[-0.005079624403207431,2.0004934129914216]},{"inlier-noise":0.021834182594597705,"outlier-noise":12.198000360301439,"prob-outlier":0.054360378346622315,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[0.025311034626876887,2.0008564257941894]},{"inlier-noise":0.0643767352077578,"outlier-noise":11.356354030256911,"prob-outlier":0.06220582369052091,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[0.04246815615866904,1.993196027714258]},{"inlier-noise":0.006800564885112691,"outlier-noise":10.642626763918537,"prob-outlier":0.0482429003450712,"outlier-designations":[false,false,false,false,false,false,false,false,false,false],"degree":1,"coeffs":[0.00376646494451649,1.99